In [ ]:
#Librairies que nous allons utiliser
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston
from keras.models import Sequential
from keras.layers import Dense, Activation
import numpy as np
import cma
import cma.purecma as purecma

In [ ]:
"""
Ici, nous avons crée des données jouets qui servirons d'entrainement pour apprendre les différentes façons
d'entrainer un modèle a réduire une fitness.
Donc notre fitnesse c'est 
"""
#Creations de données jouées 
x = np.linspace(-20,20,30).reshape((-1,1))
y = 5 * x + 3 + np.random.normal(0,5,x.shape)
y = y.reshape((-1,1))
plt.scatter(x,y)

In [ ]:
#Tracage d'une droite dans un le nuage de point
"""
Supposons que l'on ait trouvé un modèle w = a,b tel que la prédiction serrait égale y = ax + b.
Pour le tester on pourra utiliser cette fonction.
Remarque : w doit être un vecteur colonne et x un vecteur ligne pour que le produit sur les différents x
donne les prédictions du modèle.
"""

def predictions(w,x):
    ypredict = np.dot(x,w.T)
    return ypredict
    
"""
Cette fonction ajoute simple une colonne de 1 pour que chaque produit de x par un w donne x0 * a + b *1 c'est a
a dire a*x0+b.
"""
def adapt(X):
    return np.hstack((X,np.ones((X.shape[0],1))))  

#D'abord on affiche le nuage
plt.scatter(x,y)
#Ensuite on ajoute le tracé
ypred = predictions(np.array([5,2]),adapt(x)) #Donc la je test a = 5, b = 2 parce que je les connaits
plt.plot(x,ypred)
plt.show()

In [ ]:
#Résolution direct d'une regression linéaire
"""
Donc la l'objectif clairement c'est de retourver les coefficients de la droite de regression linéaire en utilisant
une résolution algébrique.
Rappel : 
    - Résoudre un systeme Xw = B reviens a trouver x c'est a dire A^-1*B (donc calculer l'inverse de A par B)
    - En Python, pour calculer l'inverse on fait : np.linalg.pinv(A) (pseudo-inversion)
"""
w = np.dot(np.linalg.pinv(adapt(x).T.dot(adapt(x))),adapt(x).T.dot(y))
print("Donc le modèle qu'on trouve est : ",w.T)
plt.scatter(x,y)
#Calcul des prédictions
ypred = predictions(w.T,adapt(x))
#Tracage du modèle : 
plt.plot(x,ypred)
plt.show()

### Résolution par descente de gradient
Le gradient d'une fonction de plusieurs paramètres correspond au vecteur des dérivées suivant les différents paramètres, il indique le sens dans lequel la fonction croit.
Par exemple : 
Soit f(x,y,z) = 3x + 8y + 3z² + 6 le gradient est 
$ \nabla $f(x,y,z) = (3,8,6z) 
et donc le gradient donne pour un point donné $\nabla$f(4,5,2) = (3,8,12).
Donc pour un coût égale a la différence entre la prédiction et la valeur réelle au carré le calcul se fait en version matricielle comme suit :

Cout des prédictions : C'est la différence entre la prédiction et la valeur qui donne un vecteur Xw - y

Donc le cout est $(Xw-y)^T * (Xw-y)$ (multiplier une ligne par une colonne sur un papier pour s'en convaincre)

Et sa dérivée est la dérivée selon w qui donne : 2X(Xw-y) et il suffit de remplacer n'importe quel w pour avoir le vecteur vers lequel ce coût augmente (et par inversion celui vers lequel le coût diminue).


In [ ]:
def cost(X,y,w):
    return np.dot(X,w)- y

def gradient(X,y,w):
    return np.dot(2*X.T,np.dot(X,w)- y)


x = np.linspace(-20,20,30).reshape((-1,1))
y = 5 * x + 3 + np.random.normal(0,5,x.shape)
y = y.reshape((-1,1))

#J'initialise les poids aléatoirement
w = np.zeros((1,2)) #1 Ligne, Deux colonnes donc un vecteur [a,b]
print(w)
#On fixe un nombre d'itérations
nb_iter = 10
x_a = adapt(x)
alpha = 1e-5
w = w.T
for i in range(10000):
    #J'affiche le coût 
    #Je met a jour les poids en suivant l'inverse du gradient
    w = w - alpha * gradient(x_a,y,w)
print("w = ",w)
w = np.dot(np.linalg.pinv(x_a.T.dot(x_a)),x_a.T.dot(y))
print("Et le modèle qu'on trouve par résolution directe est : ",w.T)

### Modéle plus compliqué
Une fois qu'on a compris une façon d'apprendre a paramétrer un modèle pour réduire un objectif (donné) il faut garder a l'esprit que le modèle linéaire est simplement un exemple de modèles.
On peut facilement l'etendre de plusieurs façons : 

In [ ]:
"""
Par exemple si on ajoute une colonne a X correspondant a x² en plus de la colonne de 1 et qu'on calcule 
un w = [a,b,c], Xw donne ax² + bx + 1 donc c'est un genre de régression quadratique.
Et ça permet d'apprendre une forme de parabole
"""
x = np.linspace(-20,20,30).reshape((-1,1))
y = 5 * x**2 + 6*x + 3 + np.random.normal(0,30,x.shape)
y = y.reshape((-1,1))
plt.scatter(x,y)

In [ ]:

def adapt_quad(X):
    return np.hstack((X**2,X,np.ones((X.shape[0],1))))  

w = np.dot(np.linalg.pinv(adapt_quad(x).T.dot(adapt_quad(x))),adapt_quad(x).T.dot(y))
print("Donc le modèle qu'on trouve est : ",w.T)
plt.scatter(x,y)
#Calcul des prédictions
ypred = predictions(w.T,adapt_quad(x))
#Tracage du modèle : 
plt.plot(x,ypred)
plt.show()

In [ ]:
x = np.linspace(-5,+5,30).reshape((-1,1))
y = np.sin(x) 
y = y.reshape((-1,1))

def adapt_quad(X):
    return np.hstack((X**2,X,np.ones((X.shape[0],1))))  

w = np.dot(np.linalg.pinv(adapt_quad(x).T.dot(adapt_quad(x))),adapt_quad(x).T.dot(y))
print("Donc le modèle qu'on trouve est : ",w.T)
plt.scatter(x,y)
#Calcul des prédictions
ypred = predictions(w.T,adapt_quad(x))
#Tracage du modèle : 
plt.plot(x,ypred)
plt.show()

In [ ]:
x = np.linspace(-5,+5,30).reshape((-1,1))
y = np.sin(x) 
y = y.reshape((-1,1))

def adapt_quad(X):
    return np.hstack((X**3,X**2,X,np.ones((X.shape[0],1))))  

w = np.dot(np.linalg.pinv(adapt_quad(x).T.dot(adapt_quad(x))),adapt_quad(x).T.dot(y))
print("Donc le modèle qu'on trouve est : ",w.T)
plt.scatter(x,y)
#Calcul des prédictions
ypred = predictions(w.T,adapt_quad(x))
#Tracage du modèle : 
plt.plot(x,ypred)
plt.show()

In [ ]:
x = np.linspace(-5,+5,30).reshape((-1,1))
y = np.sin(x) 
y = y.reshape((-1,1))

def adapt_quad(X):
    return np.hstack((X**5,X**4,X**3,X**2,X,np.ones((X.shape[0],1))))  

w = np.dot(np.linalg.pinv(adapt_quad(x).T.dot(adapt_quad(x))),adapt_quad(x).T.dot(y))
print("Donc le modèle qu'on trouve est : ",w.T)
plt.scatter(x,y)
#Calcul des prédictions
ypred = predictions(w.T,adapt_quad(x))
#Tracage du modèle : 
plt.plot(x,ypred)
plt.show()

In [ ]:
x = np.linspace(-10,+10,30).reshape((-1,1))
y = np.sin(x) 
y = y.reshape((-1,1))

def adapt_quad(X):
    return np.hstack((X**7,X**6,X**5,X**4,X**3,X**2,X,np.ones((X.shape[0],1))))  

w = np.dot(np.linalg.pinv(adapt_quad(x).T.dot(adapt_quad(x))),adapt_quad(x).T.dot(y))
print("Donc le modèle qu'on trouve est : ",w.T)
plt.scatter(x,y)
#Calcul des prédictions
ypred = predictions(w.T,adapt_quad(x))
#Tracage du modèle : 
plt.plot(x,ypred)
plt.show()

In [ ]:
x = np.linspace(-10,+10,30).reshape((-1,1))
y = np.sin(x) 
y = y.reshape((-1,1))

def adapt_quad(X):
    return np.hstack((X**13,X**12,X**11,X**10,X**9,X**8,X**7,X**6,X**5,X**4,X**3,X**2,X,np.ones((X.shape[0],1))))  

w = np.dot(np.linalg.pinv(adapt_quad(x).T.dot(adapt_quad(x))),adapt_quad(x).T.dot(y))
print("Donc le modèle qu'on trouve est : ",w.T)
plt.scatter(x,y)
#Calcul des prédictions
ypred = predictions(w.T,adapt_quad(x))
#Tracage du modèle : 
plt.plot(x,ypred)
plt.show()

Comme vous le voyez on peut apprendre mais les capacités de généralisations sont toujours trés limités, ici la solution évidente une fois qu'on constate que c'est un sinus est d'utiliser simplement la réciproque c'est a dire $sin^{-1}$ afin de repasser au cas linéaire

# Démystifier les Réseaux de Neurones

### Un Perceptron 
Enfaite un perceptron c'est juste une regression linéaire qui en sortie passe par une fonction d'activation,
En l'occurrent si la fonction d'activation est la fonction identité, c'est exactement une regression linéaire.
Ce qu'il faut comprendre c'est que chaque neurone dispose d'une matrice de pondérations pour les synapses en entrée 
![40% center](nn1.png)

Avant de commencer a expérimenter, on importe un dataset, on en profite également pour importer une fonction de
coûts implémentée par Scikit-Learn qui calcule les moindres carrés , ~~la flemme de la coder~~ les détails de son
implémentation sont laissés en exercice au lecteur

In [ ]:

X, y = load_boston(return_X_y=True)
print(X.shape)
print(y.shape)

In [ ]:
"""
Donc pour rappel un réseau de neurones se compose de : 
    - Une matrice de poids pour les synapses en entrée 
    - Une matrice de biais
    - Une fonction d'activation qui détermine la sortie en fonction d'une combinaison linéaire de l'entrée.
Calculer la sortie d'un neurone (ou d'un réseau de neurones) se fait en utilisant l'algorithme de feed forward. 
"""

weights = np.zeros((13,1)) #On met les weights a zéro par exemple.
biais = np.zeros(1)

sigm = lambda x:1/(1 + np.exp(-x))
identity = lambda x:x

def feed_forward(x,*neurone):
    w,biais,activation_func = neurone
    y = np.dot(x,w) + biais
    y = activation_func(y)
    return y

In [ ]:
"""
Imaginons que la par exemple on entraine nos pondérations par résolution directe des moindres carrés
"""
weights = np.dot(np.linalg.pinv(adapt(X).T.dot(adapt(X))),adapt(X).T.dot(y))
print("Weights = ",weights)
"""
A ce moment la la prédiction vas normalement être plus précise
Remarque : si vous comptez vous remarquerez qu'il y'a 14 weights et qu'il devrait y'en avoir 13 je vous laisse
refléchir a pourquoi. 
"""

"""
Réponse : 
"""
weights , biais = weights[:-1],weights[-1]
"""
Affichage du résultat
"""
print("prediction : ",feed_forward(X[0],weights,biais,identity), " la vraie valeur est : ", y[0])
print("Erreurs globales : ", mean_squared_error(feed_forward(X,weights,biais,identity),y))

### Keras : Un exemple de librairies pour les réseaux de neurones 
Ici je vous présente Keras une librairie utilisée pour manipuler les réseaux de neurones multi-couches

In [ ]:
"""
Cette classe contient une représentation plus complète d'un réseau de neurones multicouches.
"""
from keras.models import Sequential
from keras.layers import Dense, Activation

x = np.linspace(-200,200,1000).reshape((-1,1))
y = 5 * x + 3 + np.random.normal(0,5,x.shape)

model = Sequential([
    Dense(1, input_shape=(1,)),
    Activation('linear'),
    Dense(10,),
    Activation('linear'),
    Dense(1,),
    Activation('linear'),
])
model.compile(optimizer='rmsprop',
              loss='mse')
# Train the model, iterating on the data in batches of 32 samples
model.fit(x, y, epochs=100, batch_size=32)
#Calcul des prédictions
y_pred = model.predict(x)
#Tracage du modèle : 
plt.plot(x,y_pred)

x = np.linspace(-200,200,50).reshape((-1,1))
y = 5 * x + 3 + np.random.normal(0,5,x.shape)
plt.scatter(x,y)
plt.show()

On va tester ça sur l'exemple de tout a l'heure a présent : 

In [ ]:
X, y = load_boston(return_X_y=True)
model = Sequential([
    Dense(1, input_shape=(13,)),
    Activation('linear'),
])
model.compile(optimizer='rmsprop',
              loss='mse')
model.fit(X, y, epochs=10000, batch_size=100)

On remarque que ça prend quand même pas mal de temps, et que c'est moins précis qu'une résolution directe alors pourquoi s'en servir plûtot que la résolution directe ?

In [ ]:
X, y = load_boston(return_X_y=True)
model = Sequential([
    Dense(1, input_shape=(13,)),
    Activation('linear'),
    Dense(64,),
    Activation('relu'),
    Dense(64,),
    Activation('relu'),
    Dense(1,),
])
model.compile(optimizer='rmsprop',
              loss='mse')
model.fit(X, y, epochs=5000, batch_size=100)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation

x = np.linspace(-10,10,1000).reshape((-1,1))
y = np.sinh(x) + np.random.normal(0,100,x.shape)

w = np.dot(np.linalg.pinv(adapt(x).T.dot(adapt(x))),adapt(x).T.dot(y))
print("Donc le modèle qu'on trouve est : ",w.T)
plt.scatter(x,y)
#Calcul des prédictions
ypred = predictions(w.T,adapt(x))

#Tracage du modèle : 
plt.scatter(x,y,color="blue")
plt.plot(x,ypred,color = "red")

plt.show()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation

x = np.linspace(-5,5,1000).reshape((-1,1))
y = np.sinh(x) + np.random.normal(0,5,x.shape)

model = Sequential([
    Dense(1, input_shape=(1,)),
    Dense(64,),
    Activation('relu'),
    Dense(64,),
    Activation('relu'),
    Dense(1,),
])
model.compile(optimizer='rmsprop',
              loss='mse')
# Train the model, iterating on the data in batches of 32 samples
model.fit(x, y, epochs=100, batch_size=100)
#Calcul des prédictions
y_pred = model.predict(x)
#Tracage du modèle : 

plt.scatter(x,y)
plt.plot(x,y_pred,color="red")

plt.show()

### Manipuler un multi-couche en flat

In [ ]:
def sigmoid(x):
    return 1./(1 + np.exp(-x))

def tanh(x):
    return np.tanh(x)

class SimpleNeuralControllerNumpy():
    def __init__(self, n_in, n_out, n_hidden_layers=2, n_neurons_per_hidden=5, params=None):
        self.dim_in = n_in
        self.dim_out = n_out
        # if params is provided, we look for the number of hidden layers and neuron per layer into that parameter (a dicttionary)
        if (not params==None):
            if ("n_hidden_layers" in params.keys()):
                n_hidden_layers=params["n_hidden_layers"]
            if ("n_neurons_per_hidden" in params.keys()):
                n_neurons_per_hidden=params["n_neurons_per_hidden"]
        self.n_per_hidden = n_neurons_per_hidden
        self.n_hidden_layers = n_hidden_layers
        self.weights = None 
        self.n_weights = None
        self.init_random_params()
        self.out = np.zeros(n_out)
        #print("Creating a simple mlp with %d inputs, %d outputs, %d hidden layers and %d neurons per layer"%(n_in, n_out,n_hidden_layers, n_neurons_per_hidden))
    def init_random_params(self):
        if(self.n_hidden_layers > 0):
            self.weights = [np.random.random((self.dim_in,self.n_per_hidden))] # In -> first hidden
            self.bias = [np.random.random(self.n_per_hidden)] # In -> first hidden
            for i in range(self.n_hidden_layers-1): # Hidden -> hidden
                self.weights.append(np.random.random((self.n_per_hidden,self.n_per_hidden)))
                self.bias.append(np.random.random(self.n_per_hidden))
            self.weights.append(np.random.random((self.n_per_hidden,self.dim_out))) # -> last hidden -> out
            self.bias.append(np.random.random(self.dim_out))
        else:
            self.weights = [np.random.random((self.dim_in,self.dim_out))] # Single-layer perceptron
            self.bias = [np.random.random(self.dim_out)]
        self.n_weights = np.sum([np.product(w.shape) for w in self.weights]) + np.sum([np.product(b.shape) for b in self.bias])

    def get_parameters(self):
        """
        Returns all network parameters as a single array
        """
        flat_weights = np.hstack([arr.flatten() for arr in (self.weights+self.bias)])
        return flat_weights

    def set_parameters(self, flat_parameters):
        """
        Set all network parameters from a single array
        """
        i = 0 # index
        to_set = []
        self.weights = list()
        self.bias = list()
        if(self.n_hidden_layers > 0):
            # In -> first hidden
            w0 = np.array(flat_parameters[i:(i+self.dim_in*self.n_per_hidden)])
            self.weights.append(w0.reshape(self.dim_in,self.n_per_hidden))
            i += self.dim_in*self.n_per_hidden
            for l in range(self.n_hidden_layers-1): # Hidden -> hidden
                w = np.array(flat_parameters[i:(i+self.n_per_hidden*self.n_per_hidden)])
                self.weights.append(w.reshape((self.n_per_hidden,self.n_per_hidden)))
                i += self.n_per_hidden*self.n_per_hidden
            # -> last hidden -> out
            wN = np.array(flat_parameters[i:(i+self.n_per_hidden*self.dim_out)])
            self.weights.append(wN.reshape((self.n_per_hidden,self.dim_out)))
            i += self.n_per_hidden*self.dim_out
            # Samefor bias now
            # In -> first hidden
            b0 = np.array(flat_parameters[i:(i+self.n_per_hidden)])
            self.bias.append(b0)
            i += self.n_per_hidden
            for l in range(self.n_hidden_layers-1): # Hidden -> hidden
                b = np.array(flat_parameters[i:(i+self.n_per_hidden)])
                self.bias.append(b)
                i += self.n_per_hidden
            # -> last hidden -> out
            bN = np.array(flat_parameters[i:(i+self.dim_out)])
            self.bias.append(bN)
            i += self.dim_out
        else:
            n_w = self.dim_in*self.dim_out
            w = np.array(flat_parameters[:n_w])
            self.weights = [w.reshape((self.dim_in,self.dim_out))]
            self.bias = [np.array(flat_parameters[n_w:])]
        self.n_weights = np.sum([np.product(w.shape) for w in self.weights]) + np.sum([np.product(b.shape) for b in self.bias])
    
    def predict(self,x):
        """
        Propagage
        """
        if(self.n_hidden_layers > 0):
            #Input
            a = np.matmul(x,self.weights[0]) + self.bias[0]
            y = sigmoid(a)
            # hidden -> hidden
            for i in range(1,self.n_hidden_layers-1):
                a = np.matmul(y, self.weights[i]) + self.bias[i]
                y = sigmoid(a)
            # Out
            a = np.matmul(y, self.weights[-1]) + self.bias[-1]
            out = tanh(a)
            return out
        else: # Simple monolayer perceptron
            return tanh(np.matmul(x,self.weights[0]) + self.bias[0])

In [ ]:
Network = SimpleNeuralControllerNumpy(10, 1, n_hidden_layers=2, n_neurons_per_hidden=5, params=None)
Network.init_random_params()
print(Network.get_parameters())